In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from PIL import Image
from matplotlib.pyplot import imshow
import json

#import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

print("finish importing")

# Prepare Data

In [ ]:
'''
The file structure of the dataset
coco (DATA_ROOT)
    L modified_train2017.json
    L modified_val2017.json
    L image/
'''
DATA_ROOT = './coco'

In [ ]:
register_coco_instances(
    "modify_coco_train", 
    {}, 
    os.path.join( DATA_ROOT, "modified_train2017.json"), 
    os.path.join( DATA_ROOT, "image")
)
#register_coco_instances("modify_coco_val"  , {}, f"{DATA_ROOT}/jmodified_val2017.json"  , f"{DATA_ROOT}/image")

In [ ]:
with open('./modified_category.json', 'r') as f:
    NUM_CLASSES = len(json.load(f))
print(f"NUM_CLASSES = {NUM_CLASSES}")

# Prepare for Training

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file( model_zoo.get_config_file(  "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
"""
Model Zoo Link: 
https://github.com/facebookresearch/detectron2/blob/master/
MODEL_ZOO.md#coco-instance-segmentation-baselines-with-mask-r-cnn
"""
cfg.DATASETS.TRAIN = ("modify_coco_train",)
cfg.DATASETS.TEST  = ()
# Detectron default 4
cfg.DATALOADER.NUM_WORKERS = 4
# Detectron default 40000
cfg.SOLVER.MAX_ITER = 160_000
'''
Detectron default 
Base Learning rate 0.001
GAMMA              0.1 
STEP               (30_000,)
WEIGHT DECAY       0.000_1
MOMENTUM           0.9
    GAMMA : Learning rate decay factor
    STEPS: num of iter for learning rate decay by gamma
   
MASK RCNN PAPER : https://arxiv.org/pdf/1703.06870.pdf
    Base LR 0.02
    decay by 10 @ 120k/160k
    weight decay 0.000_1
    momentum 0.9
    
    Cityscapes finetuning 
        Base LR 0.001
        decay by 10 @ 18k/24k
    
    update baseline
        Base LR 0.001
        decay by 10 @ 120k,160k/180k
    
    Benefit form deeper model
'''   
cfg.SOLVER.BASE_LR      = 0.001  
cfg.SOLVER.GAMMA        = 0.1 
cfg.SOLVER.STEPS        = (120_000,)
cfg.SOLVER.WEIGHT_DECAY = 0.000_1
cfg.SOLVER.MOMENTUM     = 0.9

#   ROI_HEADS.BATCH_SIZE_PER_IMAGE * SOLVER.IMS_PER_BATCH
#   E.g., a common configuration is: 512 * 16 = 8192
# Detectron default 16
cfg.SOLVER.IMS_PER_BATCH = 32
# Detectron default 512
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 2048

# Number of classes 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES 

# Confident Level
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold

cfg.OUTPUT_DIR = './model'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#cfg.dump()

In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
final_model = build_model(cfg)

checkpointer = DetectionCheckpointer(final_model, save_dir="model")
checkpointer.save("save_final_model") 

# secondary save cfg as pickle
import pickle
with open('model_cfg.pickle' , 'wb') as f:
    pickle.dump(cfg,f)


# Check the Model on Robosuite Example 

In [ ]:
## load the model and the weight
"""
MODEL_ROOT
    L model_cfg.pickle
    L {cfg.OUTPUT_DIR}
        L model_final.pth
"""
MODEL_ROOT = './'
cfg = {}
with open('model_cfg.pickle' , 'rb') as f:
    cfg = pickle.load(f)

print(cfg.OUTPUT_DIR)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
pic = np.asarray(Image.open('name_0.png'))
imshow(pic)
#pic = pic.transpose((2,0,1))
print(pic.shape)

In [ ]:
outputs = predictor(pic)
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
print(outputs['instances'])

In [1]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(pic[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
imshow(out.get_image()[:, :, ::-1])
im = Image.fromarray(out)
out.save('output_name_0.jpg')

NameError: name 'Visualizer' is not defined